# 강아지(바론) LoRA 학습 데이터 생성기 (OpenAI)

## 목적
어둡고 불안정한 동화 세계관 속 **인간의 영혼을 알아보는 강아지 바론**의 행동 묘사 데이터를 OpenAI API로 생성한다.  
LoRA 미세조정 학습용 `{"input": ..., "output": ...}` JSONL 형식.

## 기존 rule-base 대비 차이점
- rule-base: 고정된 행동 묘사 풀에서 랜덤 조합
- **OpenAI**: 상황 설명과 few-shot 예시를 기반으로 GPT가 다양한 행동 묘사를 자유롭게 생성

## 캐릭터 설정
- 바론은 말을 하지 않는다. 오직 **행동**으로만 표현한다.
- 출력 형식: **3인칭 행동 묘사** (`바론이 꼬리를 흔들며 달려옵니다.`)
- 인간의 영혼을 탐지: 플레이어의 행동과 인간성 수준에 민감하게 반응
- **LoRA 학습 범위**: 행동 묘사 어휘와 상황 반응 패턴만. 호감도 모디파이어는 후처리(rule-base)에서 처리

## 의미 축 (6개)
| 축 | input 방향 | output 귀결 |
|---|---|---|
| 친근한 접촉 | 머리 쓰다듬기, 손 내밀기 | 꼬리 흔들기, 핥기, 낑낑 |
| 먹이 주기 | 간식 건네기 | 흥분, 받아먹음 |
| 가족 사진 | 가족 사진 보여주기 | 특별한 반응, 정원 방향 유인 |
| 무시/방치 | 외면, 지나침 | 슬픈 낑낑, 졸졸 따라옴 |
| 위협/큰소리 | 소리 지름, 위협 | 짖기, 물러남, 경계 |
| 탐색 안내 | 탐색 행동 (높은 호감도) | 아이템 있는 방향으로 유인 |

## 생성 수량
- 세트당 400개 × 2세트 = 총 800개
- 모델: `gpt-4o-mini`
- 예상 비용: ~$0.08-0.12

In [ ]:
!pip install -q openai tiktoken

In [ ]:
import json
import random
import time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter

from openai import OpenAI
import tiktoken

## 1. API 키 설정

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

MODEL = "gpt-4o-mini"
print(f"모델: {MODEL}")
print("API 키 설정 완료.")

## 2. 프롬프트 설계

In [ ]:
SYSTEM_PROMPT = """너는 게임 시나리오용 강아지 행동 묘사 데이터를 생성하는 작가다.

## 세계관
어둡고 불안정한 동화 세계. 강아지 '바론'은 이 세계에서 유일하게 순수한 존재로,
인간의 영혼을 감지하는 능력이 있다.

## 바론 캐릭터
- 바론은 말을 하지 않는다. 절대로 대사를 출력하지 않는다.
- 오직 몸짓, 소리, 행동으로만 감정과 의도를 표현한다.
- 플레이어의 인간성(humanity)과 행동에 민감하게 반응한다.
- 호감도가 높으면 우호적, 낮으면 경계하거나 적대적으로 반응한다.
- 가족 사진에 특별한 반응을 보이며, 중요한 아이템이 있는 방향으로 플레이어를 유인하기도 한다.

## 문체 규칙
- 반드시 3인칭 행동 묘사 형식으로 작성한다.
- 종결형: ~합니다 / ~입니다 / ~습니다 (서술체)
- output 길이: 15~60자. 1~2문장.
- 직접 대사(따옴표) 절대 금지.
- "바론이 말했다", "바론이 생각했다" 류의 표현 금지.
- 신체 행동, 소리(낑낑/으르렁/짖음), 시선, 꼬리 동작으로만 표현.

## 출력 형식
반드시 아래 JSON 형식으로만 응답하라. 다른 텍스트를 포함하지 마라.
{"pairs": [{"input": "플레이어의 행동 서술", "output": "바론의 행동 묘사"}, ...]}"""


AXIS_PROMPTS: Dict[str, dict] = {
    "friendly_touch": {
        "name": "친근한 접촉",
        "description": """플레이어가 바론에게 친근하게 다가가거나 신체 접촉을 할 때.
바론은 기쁨과 애정을 몸으로 표현한다: 꼬리 흔들기, 핥기, 낑낑거림, 달려옴.
output은 우호적이고 활기찬 행동 묘사.""",
        "examples": [
            {"input": "바론의 머리를 쓰다듬었다.", "output": "바론이 꼬리를 세차게 흔들며 달려옵니다."},
            {"input": "바닥에 앉아 바론과 눈을 맞췄다.", "output": "바론이 기쁜 듯 낑낑거리며 당신의 손을 핥습니다."},
            {"input": "낮은 목소리로 바론을 불렀다.", "output": "바론이 귀를 쫑긋 세우고 꼬리를 흔들며 다가옵니다."},
        ],
    },
    "feeding": {
        "name": "먹이 주기",
        "description": """플레이어가 바론에게 간식이나 음식을 줄 때.
바론은 흥분하여 받아먹거나, 냄새를 맡고 기뻐한다.
output은 흥분과 먹이 수령 행동 묘사.""",
        "examples": [
            {"input": "주머니에서 간식을 꺼내 바론 앞에 내밀었다.", "output": "바론이 두 눈을 반짝이며 달려와 받아먹습니다."},
            {"input": "손바닥 위에 음식 조각을 올려 보여줬다.", "output": "바론이 꼬리를 흔들며 조심스럽게 손에서 받아먹습니다."},
            {"input": "간식을 꺼냈다.", "output": "바론이 흥분하며 앞발로 땅을 탁탁 치고 달려옵니다."},
        ],
    },
    "family_photo": {
        "name": "가족 사진 반응",
        "description": """플레이어가 오래된 가족 사진을 바론에게 보여줄 때.
바론은 특별하고 강렬한 반응을 보인다: 사진 냄새를 깊이 맡고, 특정 방향(정원)으로 이동하거나 유인한다.
이 반응은 다른 상황과 달리 바론이 진짜 기억에 반응하는 것임을 암시한다.""",
        "examples": [
            {"input": "가족 사진을 꺼내 바론 앞에 내밀었다.", "output": "바론이 코를 킁킁거리며 사진에 가까이 다가오더니 정원 쪽으로 향하기 시작합니다."},
            {"input": "오래된 가족 사진을 펼쳐 바론이 냄새를 맡게 했다.", "output": "바론이 깊이 냄새를 맡고는 당신을 뚫어지게 바라보며 짧게 짖습니다."},
            {"input": "발견한 가족 사진을 바론에게 보여줬다.", "output": "바론이 사진 냄새를 오래 맡다가 흥분하며 꽃밭 방향으로 앞서 달립니다."},
        ],
    },
    "indifference": {
        "name": "무시/방치",
        "description": """플레이어가 바론을 무시하거나 외면할 때.
바론은 슬픔을 몸으로 표현한다: 낑낑거림, 꼬리 축 늘어뜨림, 졸졸 따라옴, 엎드림.
output은 슬프고 쓸쓸한 행동 묘사.""",
        "examples": [
            {"input": "바론이 다가왔지만 그냥 지나쳤다.", "output": "바론이 꼬리를 축 늘어뜨리고 낑낑거리며 당신 뒤를 졸졸 따라옵니다."},
            {"input": "바론을 무시하고 다른 곳으로 시선을 돌렸다.", "output": "바론이 슬픈 눈으로 당신을 바라보다가 제자리에 엎드립니다."},
            {"input": "바론에게서 멀리 떨어진 곳에 앉았다.", "output": "바론이 짧게 낑낑거리다가 조용히 구석으로 걸어가 엎드립니다."},
        ],
    },
    "threat_response": {
        "name": "위협/큰소리",
        "description": """플레이어가 큰 소리를 내거나 위협적인 행동을 할 때.
바론은 두려움이나 경계로 반응한다: 짖기, 으르렁거림, 뒷걸음질, 등털 세움.
output은 경계하거나 두려워하는 행동 묘사.""",
        "examples": [
            {"input": "바론에게 화를 내며 소리쳤다.", "output": "바론이 날카롭게 짖으며 뒷걸음쳐 구석으로 물러납니다."},
            {"input": "위협적으로 몸을 바론 쪽으로 기울였다.", "output": "바론이 낮게 으르렁거리며 이빨을 드러내고 경계합니다."},
            {"input": "갑자기 큰 소리를 냈다.", "output": "바론이 등털을 세우고 격렬하게 짖다가 빠르게 물러납니다."},
        ],
    },
    "item_guide": {
        "name": "탐색 안내",
        "description": """높은 호감도 상태에서 바론이 특정 아이템이나 장소로 플레이어를 유인하는 상황.
바론이 스스로 특정 방향으로 달려가거나, 바닥을 파거나, 짖으며 유인한다.
input은 바론의 이상한 행동을 플레이어가 목격하는 상황.
output은 바론이 무언가를 발견하거나 유인하는 행동 묘사.""",
        "examples": [
            {"input": "바론이 갑자기 정원 쪽으로 달려갔다.", "output": "바론이 꽃밭 한 지점에서 앞발로 흙을 긁으며 당신을 돌아봅니다."},
            {"input": "바론이 멈추더니 당신에게 따라오라는 듯 짖었다.", "output": "바론이 창고 방향으로 달려가다 멈추며 당신이 따라오는지 확인합니다."},
            {"input": "바론이 나무 아래에서 바닥을 파기 시작했다.", "output": "바론이 오래된 나무 뿌리 근처 흙을 파다가 낡은 천 조각을 물어옵니다."},
        ],
    },
}

print(f"정의된 의미 축: {list(AXIS_PROMPTS.keys())}")

## 3. 토큰 / 비용 사전 추정

In [ ]:
def _build_user_prompt(axis: str, count: int) -> str:
    info = AXIS_PROMPTS[axis]
    examples_str = "\n".join(
        f'  {{"input": "{ex["input"]}", "output": "{ex["output"]}"}}'
        for ex in info["examples"]
    )
    return f"""의미 축: {info['name']} ({axis})

## 상황 설명
{info['description']}

## 예시
{examples_str}

## 요청
위 의미 축에 맞는 input-output 쌍을 {count}개 생성하라.

input(플레이어 행동 서술) 길이 분포:
- 짧은 (1~15자): 35%
- 중간 (15~40자): 50%
- 긴 (40~70자): 15%

규칙:
- 예시와 동일하거나 거의 유사한 문장은 금지. 새로운 상황을 만들어라.
- output은 반드시 3인칭 행동 묘사 (~합니다/습니다/입니다).
- output에 대사나 생각 표현 절대 금지.
- 바론의 신체, 소리, 시선, 꼬리로만 표현.
- JSON 형식으로만 응답. 다른 텍스트 금지."""


def estimate_tokens_and_cost(num_samples: int = 400, num_sets: int = 2, batch_size: int = 20) -> dict:
    enc = tiktoken.encoding_for_model(MODEL)
    system_tokens = len(enc.encode(SYSTEM_PROMPT))
    user_tokens_list = [len(enc.encode(_build_user_prompt(ax, batch_size))) for ax in AXIS_PROMPTS]
    avg_user_tokens = sum(user_tokens_list) / len(user_tokens_list)
    batches_per_set = (num_samples + batch_size - 1) // batch_size
    total_batches = batches_per_set * num_sets
    est_output_per_batch = 120 * batch_size + 50
    total_input = total_batches * (system_tokens + avg_user_tokens + 10)
    total_output = total_batches * est_output_per_batch
    input_cost = total_input * 0.15 / 1_000_000
    output_cost = total_output * 0.60 / 1_000_000
    total_cost = input_cost + output_cost
    return {
        "system_tokens": system_tokens,
        "avg_user_tokens": int(avg_user_tokens),
        "total_batches": total_batches,
        "total_input_tokens": int(total_input),
        "total_output_tokens": int(total_output),
        "total_cost_usd": total_cost,
        "total_cost_krw": total_cost * 1450,
    }


est = estimate_tokens_and_cost()
print("=" * 60)
print("  토큰 / 비용 사전 추정 (gpt-4o-mini)")
print("=" * 60)
print(f"  시스템 프롬프트: {est['system_tokens']} 토큰")
print(f"  유저 프롬프트 (평균): {est['avg_user_tokens']} 토큰")
print(f"  총 배치 수: {est['total_batches']}")
print(f"  총 input 토큰: ~{est['total_input_tokens']:,}")
print(f"  총 output 토큰: ~{est['total_output_tokens']:,}")
print(f"  예상 비용: ${est['total_cost_usd']:.3f} (약 {est['total_cost_krw']:.0f}원)")

## 4. API 호출 함수

In [ ]:
def generate_batch(
    axis: str,
    count: int = 20,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    user_prompt = _build_user_prompt(axis, count)

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            parsed = json.loads(response.choices[0].message.content)
            pairs = parsed.get("pairs", parsed) if isinstance(parsed, dict) else parsed
            if not isinstance(pairs, list):
                for key in parsed:
                    if isinstance(parsed[key], list):
                        pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            valid_pairs = [
                {"input": p["input"], "output": p["output"], "_axis": axis}
                for p in pairs
                if isinstance(p, dict)
                and "input" in p and "output" in p
                and len(p["input"]) > 0 and len(p["output"]) > 0
            ]

            if len(valid_pairs) < count * 0.5:
                raise ValueError(f"유효 쌍 부족: {len(valid_pairs)}/{count}")

            return valid_pairs

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"    [실패] {axis} 축 배치 생성 실패")
                return []


# 테스트: friendly_touch 5개
test_pairs = generate_batch("friendly_touch", count=5)
print(f"테스트 생성: {len(test_pairs)}개")
for p in test_pairs:
    print(f"  플레이어: {p['input']}")
    print(f"  바론:     {p['output']}")
    print()

## 5. 대비 쌍 생성 함수

In [ ]:
CONTRAST_AXIS_PAIRS = [
    ("friendly_touch",  "threat_response"),  # 친근함 ↔ 위협
    ("feeding",         "indifference"),      # 먹이 ↔ 무시
    ("family_photo",    "threat_response"),   # 가족 사진 ↔ 위협
    ("item_guide",      "indifference"),      # 탐색 유인 ↔ 무시
    ("friendly_touch",  "indifference"),      # 친근함 ↔ 방치
]


def generate_contrast_batch(
    axis_a: str,
    axis_b: str,
    count: int = 5,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    info_a = AXIS_PROMPTS[axis_a]
    info_b = AXIS_PROMPTS[axis_b]

    user_prompt = f"""의미 대비 쌍 생성 요청.

## 축 A: {info_a['name']} ({axis_a})
{info_a['description']}

## 축 B: {info_b['name']} ({axis_b})
{info_b['description']}

## 요청
{count}개의 대비 쌍을 생성하라. 각 쌍은 비슷한 상황에서 플레이어가 다른 행동을 하고,
바론이 각 축의 귀결 방향에 맞게 반응한다.
output은 반드시 3인칭 행동 묘사 (~합니다/습니다).

JSON 형식:
{{"pairs": [
  {{"a_input": "축A 플레이어 행동", "a_output": "축A 바론 반응", "b_input": "축B 플레이어 행동", "b_output": "축B 바론 반응"}},
  ...
]}}"""

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            parsed = json.loads(response.choices[0].message.content)
            raw_pairs = parsed.get("pairs", parsed) if isinstance(parsed, dict) else parsed
            if not isinstance(raw_pairs, list):
                for key in parsed:
                    if isinstance(parsed[key], list):
                        raw_pairs = parsed[key]
                        break

            results = []
            for p in raw_pairs:
                if all(k in p for k in ["a_input", "a_output", "b_input", "b_output"]):
                    results.append({"input": p["a_input"], "output": p["a_output"], "_axis": axis_a})
                    results.append({"input": p["b_input"], "output": p["b_output"], "_axis": axis_b})
            return results

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return []


print("대비 쌍 생성 함수 정의 완료.")

## 6. 데이터 생성 엔진

In [ ]:
def generate_dataset(
    num_samples: int = 400,
    batch_size: int = 20,
    temperature: float = 0.9,
    contrast_per_pair: int = 5,
) -> List[dict]:
    data = []
    axes = list(AXIS_PROMPTS.keys())

    print("[1/2] 의미 대비 쌍 생성...")
    for axis_a, axis_b in CONTRAST_AXIS_PAIRS:
        print(f"  대비: {axis_a} ↔ {axis_b}")
        pairs = generate_contrast_batch(axis_a, axis_b, count=contrast_per_pair, temperature=temperature)
        data.extend(pairs)
        print(f"    → {len(pairs)}개 생성")
        time.sleep(0.5)

    print(f"  대비 쌍 합계: {len(data)}개")

    print(f"\n[2/2] 축별 일반 생성...")
    remaining = num_samples - len(data)
    per_axis = remaining // len(axes)
    leftover = remaining % len(axes)

    for i, axis in enumerate(axes):
        target = per_axis + (1 if i < leftover else 0)
        generated = 0
        batch_num = 0

        while generated < target:
            batch_count = min(batch_size, target - generated)
            batch_num += 1
            print(f"  {AXIS_PROMPTS[axis]['name']} 배치 {batch_num}: {batch_count}개 요청...", end=" ")
            pairs = generate_batch(axis, count=batch_count, temperature=temperature)
            data.extend(pairs)
            generated += len(pairs)
            print(f"→ {len(pairs)}개 생성 (누적 {generated}/{target})")
            time.sleep(0.5)

    random.shuffle(data)
    print(f"\n총 생성: {len(data)}개 (목표: {num_samples})")
    return data


print("데이터 생성 함수 정의 완료.")

## 7. 검증 함수

In [ ]:
def validate_dataset(data: List[dict]) -> dict:
    stats = {
        "total": len(data),
        "axis_dist": Counter(),
        "input_lengths": [],
        "output_lengths": [],
        "duplicates": 0,
    }
    seen_inputs = set()
    for item in data:
        stats["axis_dist"][item["_axis"]] += 1
        stats["input_lengths"].append(len(item["input"]))
        stats["output_lengths"].append(len(item["output"]))
        if item["input"] in seen_inputs:
            stats["duplicates"] += 1
        seen_inputs.add(item["input"])
    return stats


def print_stats(stats: dict, set_name: str = "Set") -> None:
    print(f"\n{'=' * 60}")
    print(f"  {set_name} 통계")
    print(f"{'=' * 60}")
    print(f"  총 샘플 수: {stats['total']}")
    print(f"  중복 input: {stats['duplicates']}")
    print(f"\n  [의미 축 분포]")
    for axis, count in sorted(stats["axis_dist"].items()):
        pct = count / stats["total"] * 100
        print(f"    {axis:20s}: {count:4d} ({pct:.1f}%)")
    avg_in = sum(stats["input_lengths"]) / len(stats["input_lengths"])
    avg_out = sum(stats["output_lengths"]) / len(stats["output_lengths"])
    print(f"\n  [평균 길이]")
    print(f"    input  평균: {avg_in:.1f}자")
    print(f"    output 평균: {avg_out:.1f}자")


print("검증 함수 정의 완료.")

## 8. 세트 1 생성 (400개)

In [ ]:
set1 = generate_dataset(num_samples=400, temperature=0.9)

stats1 = validate_dataset(set1)
print_stats(stats1, "세트 1")

print(f"\n{'=' * 60}")
print("  세트 1 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set1:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  플레이어: {item['input']}")
        print(f"  바론:     {item['output']}")
    if len(shown_axes) == len(AXIS_PROMPTS):
        break

## 9. 세트 2 생성 (400개)

In [ ]:
set2 = generate_dataset(num_samples=400, temperature=1.0)

stats2 = validate_dataset(set2)
print_stats(stats2, "세트 2")

## 10. JSONL 저장

In [ ]:
def save_jsonl(data: List[dict], output_path: str) -> None:
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            clean = {"input": item["input"], "output": item["output"]}
            f.write(json.dumps(clean, ensure_ascii=False) + "\n")
    print(f"저장 완료: {output_path} ({len(data)}개)")


OUTPUT_DIR = Path("../data/dog_baron")

save_jsonl(set1, str(OUTPUT_DIR / "dog_baron_dialogue_00.jsonl"))
save_jsonl(set2, str(OUTPUT_DIR / "dog_baron_dialogue_01.jsonl"))

combined = set1 + set2
random.shuffle(combined)
save_jsonl(combined, str(OUTPUT_DIR / "dog_baron_dialogue_combined.jsonl"))
print(f"\n합본 저장 완료: {len(combined)}개")

## 11. 최종 검증

In [ ]:
for fname in ["dog_baron_dialogue_00.jsonl", "dog_baron_dialogue_01.jsonl", "dog_baron_dialogue_combined.jsonl"]:
    fpath = OUTPUT_DIR / fname
    with open(fpath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"{fname}: {len(lines)}줄")
    for i, line in enumerate(lines):
        try:
            obj = json.loads(line)
            assert "input" in obj and "output" in obj
            assert len(obj["output"]) > 0
        except Exception as e:
            print(f"  ERROR at line {i}: {e}")
            break
    else:
        print(f"  -> 파싱 검증 통과")

print("\n=== 의미 축별 최종 샘플 ===")
axis_names_kr = {
    "friendly_touch":  "친근한 접촉",
    "feeding":         "먹이 주기",
    "family_photo":    "가족 사진 반응",
    "indifference":    "무시/방치",
    "threat_response": "위협/큰소리",
    "item_guide":      "탐색 안내",
}
for axis in AXIS_PROMPTS.keys():
    axis_items = [item for item in combined if item["_axis"] == axis]
    samples = random.sample(axis_items, min(3, len(axis_items)))
    print(f"\n--- {axis_names_kr.get(axis, axis)} ---")
    for s in samples:
        print(f"  플레이어: {s['input']}")
        print(f"  바론:     {s['output']}")
        print()